In [ ]:
import json
import pandas as pd
import requests
import io

In [ ]:
url = "https://data.ofgl.fr/api/explore/v2.1/catalog/datasets/ofgl-base-gfp/exports/csv?select=exer%2C%20reg_name%2C%20dep_name%2C%20nat_juridique%2C%20mode_financement%2C%20gfp_qpv%2C%20epci_name%2C%20epci_code%2C%20lbudg%2C%20type_de_budget%2C%20agregat%2C%20montant_en_millions%2C%20euros_par_habitant%2C%20ptot&where=%28agregat%20%3D%20%22Encours%20de%20dette%22%20or%20agregat%20%3D%20%22Produit%20des%20cessions%20d%27immobilisations%22%20or%20agregat%20%3D%20%22Recettes%20de%20fonctionnement%22%20or%20agregat%20%3D%20%22Epargne%20de%20gestion%22%20or%20agregat%20%3D%20%22Epargne%20brute%22%20or%20agregat%20%3D%20%22Epargne%20nette%22%20or%20agregat%20%3D%20%22Remboursements%20d%27emprunts%20hors%20GAD%22%20or%20agregat%20%3D%20%22Annuit%C3%A9%20de%20la%20dette%22%29%20and%20year%28exer%29%20%3D%202023&limit=-1&timezone=UTC&use_labels=false&epsg=4326"
result = requests.get(url)
df = pd.read_csv(io.StringIO(result.content.decode('utf-8')), sep=";")

In [ ]:
df_principal = df.copy()

In [ ]:
df_principal['encours_budget_principal'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Encours de dette" else None, axis=1)
df_principal['recette_fonctionnement'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Recettes de fonctionnement" else None, axis=1)
df_principal['epargne_gestion'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne de gestion" else None, axis=1)
df_principal['epargne_brute'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne brute" else None, axis=1)
df_principal['epargne_nette'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Epargne nette" else None, axis=1)

In [ ]:
df_principal['Produit des cessions d’immobilisations'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Produit des cessions d'immobilisations" else None, axis=1)
df_principal['Remboursements d emprunts hors GAD'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Remboursements d'emprunts hors GAD" else None, axis=1)
df_principal['Annuité de la dette'] = df_principal.apply(lambda row: row['montant_en_millions'] if row['agregat'] == "Annuité de la dette" else None, axis=1)

In [ ]:
df_final = df_principal.copy()

In [ ]:
df_final = df_principal.groupby('epci_code').agg(
    {
        'exer':'first',
        'reg_name':'first',
        'dep_name':'first',
        'nat_juridique':'first',
        'mode_financement':'first',
        'gfp_qpv':'first',
        'epci_name':'first',
        'lbudg':'first',
        'type_de_budget':'first',
        'ptot':'first',
        'euros_par_habitant':'first',
        'encours_budget_principal': lambda x: x.dropna().sum(),
        'recette_fonctionnement': lambda x: x.dropna().sum(),
        'epargne_gestion': lambda x: x.dropna().sum(),
        'epargne_brute': lambda x: x.dropna().sum(),
        'epargne_nette': lambda x: x.dropna().sum(),
        'Produit des cessions d’immobilisations': lambda x: x.dropna().sum(),
        'Remboursements d emprunts hors GAD': lambda x: x.dropna().sum(),
        'Annuité de la dette': lambda x: x.dropna().sum()
    }
).reset_index()

In [ ]:
df_final['capacite_desendetemment'] = df_final.apply(lambda row: row['encours_budget_principal'] / row['epargne_brute'] if row['epargne_brute'] else None, axis=1)
df_final['taux_endetemment'] = df_final.apply(lambda row: row['encours_budget_principal'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['duree_apparente_dette'] = df_final.apply(lambda row: row['encours_budget_principal'] / row["Remboursements d emprunts hors GAD"] if row["Remboursements d emprunts hors GAD"] else None, axis=1)
df_final['taux_epargne_net'] = df_final.apply(lambda row: row['epargne_nette'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['taux_epargne_brute'] = df_final.apply(lambda row: row['epargne_brute'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['taux_epargne_gestion'] = df_final.apply(lambda row: row['epargne_gestion'] / row['recette_fonctionnement'] if row['recette_fonctionnement'] else None, axis=1)
df_final['ratio_annuité_recette_fonctionnement'] = df_final.apply(lambda row: row['epargne_nette'] / row['Annuité de la dette'] if row['Annuité de la dette'] else None, axis=1)


In [ ]:
df_final.to_csv('donnees_financieres_inter.csv')

In [ ]:
import json
import pandas as pd

def get_json_finance_inter(dataframe,epci_code, exer):
  """Retrieves data from the DataFrame and returns it as a JSON.

  Args:
    epci_code: The EPCI code to filter by.
    exer: The exer to filter by.

  Returns:
    A JSON object containing the data, or an error message if data is not found or if inputs are invalid.
  """
  try:
    exer = int(exer)  # Convert exer to integer
  except ValueError:
    return json.dumps({"error": "Invalid exer value. Please provide a valid year."})

  filtered_df = df_final[(dataframe['epci_code'] == epci_code) & (dataframe['exer'] == exer)]

  if filtered_df.empty:
    return json.dumps({"error": f"No data found for epci_code: {epci_code} and exer: {exer}"})

  data_dict = filtered_df.iloc[0].to_dict()

  # Handle potential NaN values
  for key, value in data_dict.items():
      if pd.isnull(value):
          data_dict[key] = None  # or any other desired representation for NaN

  return json.dumps(data_dict, ensure_ascii=False).encode('utf8')

test = get_json_finance_inter(df_final,242100410, 2023)
print(test.decode())  # This would print the JSON string (if data is found).